In [1]:
!pip install keybert

You should consider upgrading via the 'C:\Users\User-Pc\miniconda3\python.exe -m pip install --upgrade pip' command.



     -------------------------------------- 235.6/235.6 KB 4.8 MB/s eta 0:00:00
     ---------------------------------------- 51.1/51.1 KB 2.6 MB/s eta 0:00:00


In [2]:
from keybert import KeyBERT
import pickle
import re
from tqdm import tqdm
import pandas as pd

In [15]:
name = 'news_final.pickle'
with open('C:/Users/Notebiz003/BerTopic/news/' + name, 'rb') as f:
    data = pickle.load(f)

In [16]:
data = data.reset_index()

In [18]:
idx = []
for x in data['index']:
    tmp = x.split('/')
    if len(tmp[0]) == 1:
        tmp[0] = '0'+tmp[0]
    if len(tmp[1]) == 1:
        tmp[1] = '0'+tmp[1]
    tmp = tmp[2] + tmp[0] + tmp[1]
    idx.append(''.join(tmp))

In [19]:
data['index'] = idx
data = data.rename({'index':'일자',0:'news'},axis = 1)

In [20]:
def doc2key(cleaned_content):
    
    kw_model = KeyBERT(model='all-MiniLM-L6-v2')
    keywords = kw_model.extract_keywords(doc)
    
    n2_kwd = kw_model.extract_keywords(cleaned_content, keyphrase_ngram_range=(2, 2), stop_words='english',
                                  use_mmr=True, diversity=0.7, top_n=100)

    n1_kwd = kw_model.extract_keywords(cleaned_content, keyphrase_ngram_range=(1, 1), stop_words='english',
                                  use_mmr=True, diversity=0.7, top_n=50)
    for idx,i in enumerate(n2_kwd):
        n2_kwd[idx] = i[0]
    for idx,i in enumerate(n1_kwd):
        n1_kwd[idx] = i[0]  

    n1_kwd.extend(n2_kwd)
    kwd = n1_kwd
    return kwd

In [ ]:
tmp_dfs = []
for idx in tqdm(range(len(data['일자']))):
    news = data.iloc[idx,:]['news']
    date = data.iloc[idx,:]['일자']
    tmp_dict = dict()
    tmp = []
    for doc in news:
        cleaned_content = re.sub(r'[^\.\?\!\w\d\s]','',doc) # 문장단위로 끊기
        cleaned_content = cleaned_content.replace('\n',' ')
        cleaned_content = cleaned_content.lower()
        kwd = doc2key(cleaned_content)
        tmp.append(kwd)

    tmp_dict[date] = tmp
    tmp_df = pd.DataFrame(tmp_dict)
    tmp_df['일자'] = [date for x in range(len(tmp))]
    tmp_df = tmp_df.rename({date:'키워드'},axis = 1)
    tmp_df = tmp_df[['일자','키워드']]
    tmp_dfs.append(tmp_df)

  4%|██▊                                                                           | 39/1095 [19:23<6:11:09, 21.09s/it]

In [8]:
df = pd.concat(tmp_dfs)

In [1]:
df

NameError: name 'df' is not defined

In [8]:
data

,일자,news
0,20190801,[Newspapers have lost about 57% of their ad an...
1,20190802,[Why did this happen?\n\nPlease make sure your...
2,20190803,[Become an FT subscriber to read:\n\nLeverage ...
3,20190804,[Make informed decisions with the FT\n\nKeep a...
4,20190805,[Today Standard Chartered announced it complet...
...,...,...
1090,20220727,[Dealing with the repercussions of the multibi...
1091,20220728,[A report produced by the Alliance for Financi...
1092,20220729,"[For finance departments, the pursuit of great..."
1093,20220730,"[By Monish Anand\n\nIn India, a significant pa..."
